## gDrive 연결
> - drive 폴더가 붙은 것을 확인한다.
> - drwx------ 4 root root 4.0K Mar 22 11:21 drive

In [0]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 모듈 Install / Import 

In [0]:
# 코랩에 없는 라이브러리 설치해주기
!pip install bayesian-optimization

In [0]:
import os
import sys           
import random 
import warnings          

import numpy as np                          # 계산 라이브러리
import pandas as pd                         # 데이터 분석 라이브러리
import matplotlib.pyplot as plt             # * 그래프 이미지
import lightgbm as lgb                      # LightGBM 라이브러리 

from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정

warnings.filterwarnings("ignore")           # 경고 문구 미표시

In [0]:
"""
# 데이터가 많아 5분 정도 걸립니다.  
# 전체경로 = /content/drive/My Drive/Colab Notebooks/
# kay_suparx_learning_model/data_remake/xtra_remake_xtest_final.csv
"""

dir_base = '/content/drive/My Drive/Colab Notebooks//kay_suparx_learning_model/'

raw    = 'data_raw/'
remake = 'data_remake/'
submit = 'data_submit/'

# WAY-01 = LGBM, WAY-02 = XGBM
init_points, n_iter = (15, 75)                      
filename_submit = f"submission_0414_way01_{init_points}_{n_iter}_final.csv"

## 사용함수의 정의

In [0]:
if dir_base in sys.path:
  pass
else:
  print(f"*** '{dir_base} is inserted! ***'")
  sys.path.insert(0, dir_base )

sys.path

['/content/drive/My Drive/Colab Notebooks//kay_suparx_learning_model/',
 '',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
os.chdir(dir_base)
from _assets.modules import *
from _assets.module_lgbm_model import *
# from _assets.module_data_preps import *

In [0]:
show_ls(dir_base + remake)

False
DIR_TARGET=/content/drive/My Drive/Colab Notebooks/kay_suparx_learning_model
/content/drive/My Drive/Colab Notebooks//kay_suparx_learning_model/data_remake/
----------------------------------------
  01. df_ytrain_remake.csv
  02. xtra_remake_xtest_final.csv
  03. xtra_remake_xtrain_final.csv


## 데이터를 불러온다

In [0]:
%%time
"""
# show_infoDF_from(x_train) # *** DATA SHAPE = [ 38,872 x 1,579 ] 
# to <class 'pandas.core.frame.DataFrame'>
# show_infoDF_from(y_train) # *** DATA SHAPE = [ 38,872 x 3 ]
# to <class 'numpy.ndarray'> ...  array([1, 1, 0, ..., 0, 1, 0])
"""

x_train = pd.read_csv(dir_base + remake + 'xtra_remake_xtrain_final.csv')  
ytrain = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv')  
y_train = ytrain['winner'].values                                 

CPU times: user 4.13 s, sys: 284 ms, total: 4.41 s
Wall time: 4.5 s


In [0]:
"""  
# 1    19499
# 0    19373 ... y_train = np.array 38,872   
"""
pd.Series(y_train).value_counts()

1    19499
0    19373
dtype: int64

## LGBM Classifier 모델 적용
 -  Light Gradient Boost Method 적용 - [출처] :  YW & YY's Python, Machine & Deep Learning
 > - Light GBM: A Highly Efficient Gradient Boosting Decision Tree 논문 리뷰
 > - [Light GBM 모델 설명 및 사용법](https://greeksharifa.github.io/machine_learning/2019/12/09/Light-GBM/)

 > 다중 분류, 클릭 예측, 순위 학습 등에 주로 사용되는 Gradient Boosting Decision Tree (GBDT)는 굉장히 유용한 머신러닝 알고리즘이며, XGBoost나 pGBRT 등 효율적인 기법의 설계를 가능하게 하였다. 이러한 구현은 많은 엔지니어링 최적화를 이룩하였지만 고차원이고 큰 데이터 셋에서는 만족스러운 결과를 내지 못하는 경우도 있었다. 왜냐하면 모든 가능한 분할점에 대해 정보 획득을 평가하기 위해 데이터 개체 전부를 스캔해야 했기 때문이다. 이는 당연하게도, 굉장히 시간 소모적이다.


In [29]:
%%time
func_fixed = partial(                        # 모델과 관련없는 변수 고정
                  lgb_cv, 
                  x_data=x_train, 
                  y_data=y_train, 
                  n_splits=5, 
                  output='score',
                ) 

lgbBO = BayesianOptimization(                # 베이지안 최적화 범위 설정
      func_fixed, 
      {
          'learning_rate': (0.0001, 0.1),    # learning_rate,    범위(0.0001~0.1)
          'num_leaves': (16, 1024),          # num_leaves,       범위(16~1024)
          'n_estimators': (16, 1024),        # n_estimators,     범위(16~1024)
          'subsample': (0, 1),               # subsample,        범위(0~1)
          'colsample_bytree': (0, 1),        # colsample_bytree, 범위(0~1)
          'reg_alpha': (0, 10),              # reg_alpha,        범위(0~10)
          'reg_lambda': (0, 50),             # reg_lambda,       범위(0~50)
      }, 
      random_state=random.randrange(50000)   # 시드 고정 = 4321
  )

"""
# 처음 5회 랜덤 값으로 score 계산 후, 30회 최적화 = Original Baseline!
# lgbBO.maximize(init_points=5, n_iter=30) 
"""
lgbBO.maximize(init_points, n_iter)

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.7032   |  0.03509  |  0.03434  |  984.5    |  711.5    |  6.349    |  5.192    |  0.2539   |
|  2        |  0.6999   |  0.4272   |  0.07676  |  461.7    |  559.8    |  9.507    |  44.73    |  0.6343   |
|  3        |  0.7006   |  0.2366   |  0.07916  |  423.4    |  166.6    |  4.869    |  2.015    |  0.2579   |
|  4        |  0.6788   |  0.8498   |  0.07719  |  37.83    |  293.0    |  6.974    |  39.14    |  0.7509   |
|  5        |  0.7051   |  0.5616   |  0.07109  |  586.6    |  843.3    |  0.7208   |  45.65    |  0.3214   |
|  6        |  0.7072   |  0.5922   |  0.02191  |  687.4    |  356.4    |  5.153    |  46.61    |  0.7281   |
|  7        |  0.6988   |  0.3364   |  0.06713  |  772.1    |  591.6    |  8.717    |  15.72    |  0.3804   |
|  8      

## 모델 학습 및 검증
- Model Tuning & Evaluation

> 1. AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.
> 1. 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.
> 1. 최종 모델을 사용해 예측을 수행합니다.
> 1. 예측 결과를 submission.csv로 저장합니다.

In [0]:
params = lgbBO.max['params']

models = lgb_cv(
        params['num_leaves'], 
        params['learning_rate'], 
        params['n_estimators'], 
        params['subsample'], 
        params['colsample_bytree'], 
        params['reg_alpha'], 
        params['reg_lambda'], 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model',
    )

## TEST를 풀기위해, 학습데이터로 변경 (data_preps)
- df_xtrain_final.shape   # (38872, 1579)
- df_xtest_final.shape   # (16787, 1579)

In [0]:
"""
# 테스트용 데이터를 전처리 해서 분석 준비 ... (시간소요) 미리 저장한 화일을 불러온다
# 저장했으면 비활성 처리
# x_test, _ = data_preparation(test, answer=False)                 # [ 28,714,849 x 6 ] ... 30%
# x_test.to_csv(dir_base + remake + 'df_xtest_remake.csv')
# x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')  # [  16,787 x 27 ]
"""
x_test = pd.read_csv(dir_base + remake + 'xtra_remake_xtest_final.csv')    # *** DATA SHAPE = [ 16,787 x 27 ]


## 예측결과 Submission 화일 만들기

In [0]:
"""
# pd.read_csv('data/sample_submission.csv', index_col=0)
# sample_submission = pd.read_csv(dir_base + sub_base + "sample_submission.csv")
"""

preds = []

for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

In [33]:
show_ls(dir_base + raw)

False
DIR_TARGET=/content/drive/My Drive/Colab Notebooks/kay_suparx_learning_model/data_remake
/content/drive/My Drive/Colab Notebooks//kay_suparx_learning_model/data_raw/
----------------------------------------
  01. sample_submission.csv


In [0]:
submission = pd.read_csv(dir_base + raw + "sample_submission.csv") 


submission['winner'] = pred
submission.to_csv(dir_base + submit + filename_submit, index=False)

# 결과 확인하기
- 3월 데이콘 대회 홈페이지 = https://bit.ly/39bqWVg
- 결과 제출하기 = https://dacon.io/competitions/official/235583/mysubmission/

In [35]:
df_1 = submission
df_1

,game_id,winner
0,38872,0.446142
1,38873,0.587099
2,38874,0.401160
3,38875,0.186928
4,38876,0.634290
...,...,...
16782,55654,0.697729
16783,55655,0.188369
16784,55656,0.762855
16785,55657,0.522746
